In [1]:
import os
import yaml

# Load the original and desired labels from the YAML files
with open('./data.yaml', 'r') as file:
    original_data = yaml.safe_load(file)

with open('./desired.yaml', 'r') as file:
    desired_data = yaml.safe_load(file)

# Extract labels from YAML files as dictionaries with integer keys
original_label_dict = {int(k): v for k, v in original_data['names'].items()}
desired_label_dict = {int(k): v for k, v in desired_data['names'].items()}

# Create reverse mappings: label name -> index
original_name_to_index = {v: k for k, v in original_label_dict.items()}
desired_name_to_index = {v: k for k, v in desired_label_dict.items()}

# Create a mapping from original label indices to desired label indices based on label names
label_index_mapping = {}
for orig_index, label_name in original_label_dict.items():
    if label_name in desired_name_to_index:
        desired_index = desired_name_to_index[label_name]
        label_index_mapping[orig_index] = desired_index
    else:
        # I skip labels not present in desired.yaml
        print(f"Label '{label_name}' (index {orig_index}) not in desired labels. It will be excluded.")
        # Optionally, map to a default index or handle as needed
        # label_index_mapping[orig_index] = default_index

# Directory containing annotation files
annotations_dir = './labels/train/'

# Get all .txt files in the annotations directory
annotation_files = [os.path.join(annotations_dir, f) for f in os.listdir(annotations_dir) if f.endswith('.txt')]

def remap_labels(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    remapped_lines = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) < 5:
            # Line doesn't have enough parts, skip it
            print(f"Invalid line in file '{file_path}': {line.strip()}")
            continue
        label_index = int(parts[0])
        if label_index in label_index_mapping:
            new_label_index = label_index_mapping[label_index]
            parts[0] = str(new_label_index)
            remapped_line = " ".join(parts)
            remapped_lines.append(remapped_line)
        else:
            # If the label index is not in the mapping, exclude the line or handle as needed
            print(f"Label index {label_index} not in mapping for file '{file_path}'. Line will be excluded.")
            # Optionally, keep the original line or map to a default index
            # remapped_lines.append(line.strip())

    # Write the remapped lines back to the file
    with open(file_path, 'w') as file:
        file.write("\n".join(remapped_lines))

# Apply remapping to all annotation files
for file_path in annotation_files:
    remap_labels(file_path)

print("Label remapping completed.")

Label 'RLY' (index 13) not in desired labels. It will be excluded.
Label 'RMY' (index 14) not in desired labels. It will be excluded.
Label 'RLR' (index 16) not in desired labels. It will be excluded.
Label 'RMR' (index 17) not in desired labels. It will be excluded.
Label 'SR' (index 23) not in desired labels. It will be excluded.
Label 'PMPL' (index 36) not in desired labels. It will be excluded.
Label 'GRLG' (index 52) not in desired labels. It will be excluded.
Label remapping completed.
